In [3]:
import numpy as np
from qiskit.quantum_info import Statevector, Operator
from qiskit import QuantumCircuit, QuantumRegister
from scipy.linalg import solve
from qiskit import QuantumRegister as qr
from qiskit import QuantumCircuit as qc

def find_preparation_gate(target_amplitudes):
    """
    Find the unitary matrix that transforms |0...0⟩ to the desired state.
    
    Args:
        target_amplitudes (list): List of complex amplitudes for the target state
    
    Returns:
        numpy.ndarray: The unitary matrix that performs the transformation
    """
    # Convert to numpy array and normalize
    target_state = target_amplitudes / np.linalg.norm(target_amplitudes)
    
    n_qubits = int(np.log2(len(target_state)))
    dim = 2**n_qubits
    
    # Create initial state |0...0⟩
    initial_state = np.zeros(dim)
    initial_state[0] = 1
    
    # Create a basis for the orthogonal complement
    basis = np.eye(dim)
    v1 = target_state
    
    # Gram-Schmidt process to create orthonormal basis
    Q = np.zeros((dim, dim), dtype=complex)
    Q[:, 0] = v1
    
    for i in range(1, dim):
        v = basis[:, i]
        for j in range(i):
            v = v - np.dot(Q[:, j].conj(), v) * Q[:, j]
        if np.linalg.norm(v) > 1e-10:  # Check if vector is non-zero
            v = v / np.linalg.norm(v)
        Q[:, i] = v
    
    # Construct unitary matrix
    U = Q
    
    return U

# Create quantum circuit
#qr_1 = QuantumRegister(5)
#qc_1 = QuantumCircuit(qr_1)

#qc_1.h(list(range(5)))

# Define target state amplitudes
#amplitudes = Statevector.from_instruction(qc_1).data

#print(amplitudes)

# Define single Hadamard gate
H = 1/np.sqrt(2) * np.array([[1, 1],
                            [1, -1]])

# Calculate tensor product of 5 H gates
# Using np.kron repeatedly
U = H
for _ in range(4):  # We need to do it 4 more times
    U = np.kron(U, H)

# Find the unitary matrix
#U = find_preparation_gate(amplitudes)

U

array([[ 0.1767767,  0.1767767,  0.1767767, ...,  0.1767767,  0.1767767,
         0.1767767],
       [ 0.1767767, -0.1767767,  0.1767767, ..., -0.1767767,  0.1767767,
        -0.1767767],
       [ 0.1767767,  0.1767767, -0.1767767, ...,  0.1767767, -0.1767767,
        -0.1767767],
       ...,
       [ 0.1767767, -0.1767767,  0.1767767, ...,  0.1767767, -0.1767767,
         0.1767767],
       [ 0.1767767,  0.1767767, -0.1767767, ..., -0.1767767,  0.1767767,
         0.1767767],
       [ 0.1767767, -0.1767767, -0.1767767, ...,  0.1767767,  0.1767767,
        -0.1767767]])

In [4]:
"""Number of qubits."""
N: int = 5

"""How much optimization to perform on the circuits.
   Higher levels are more optimized, but take longer to transpile.

   * 0 = No optimization
   * 1 = Light optimization
   * 2 = Heavy optimization
   * 3 = Heavier optimization
"""
OPTIMIZATION_LEVEL: int = 2

"""Set of m nonnegative integers to search for using Grover's algorithm (i.e. TARGETS in base 10)."""
#SEARCH_VALUES: set[int] = { 9, 0, 3 }
SEARCH_VALUES: set[int] = {5 }

"""Set of m N-qubit binary strings representing target state(s) (i.e. SEARCH_VALUES in base 2)."""
TARGETS: set[str] = { f"{s:0{N}b}" for s in SEARCH_VALUES }

UGATE: np.array = U

"""N-qubit quantum register."""
QUBITS: qr = qr(N, "qubit")

def flip(target: str, qc: qc, qubit: str = "0") -> None:
    """Flips qubit in target state.

    Args:
        target (str): Binary string representing target state.
        qc (qc): Quantum circuit.
        qubit (str, optional): Qubit to flip. Defaults to "0".
    """
    for i in range(len(target)):
        if target[i] == qubit:
            qc.x(i) # Pauli-X gate

def oracle(targets: set[str] = TARGETS, name: str = "Oracle") -> qc:
    """Mark target state(s) with negative phase.

    Args:
        targets (set[str]): Binary string(s) representing target state(s). Defaults to TARGETS.
        name (str, optional): Quantum circuit's name. Defaults to "Oracle".

    Returns:
        qc: Quantum circuit representation of oracle.
    """
    # Create N-qubit quantum circuit for oracle
    oracle = qc(QUBITS, name = name)

    for target in targets:
        # Reverse target state since Qiskit uses little-endian for qubit ordering
        target = target[::-1]
        
        # Flip zero qubits in target
        flip(target, oracle, "0")

        # Simulate (N - 1)-control Z gate
        oracle.h(N - 1)                       # Hadamard gate
        oracle.mcx(list(range(N - 1)), N - 1) # (N - 1)-control Toffoli gate
        oracle.h(N - 1)                       # Hadamard gate

        # Flip back to original state
        flip(target, oracle, "0")

    return oracle


In [5]:
def diffuser(name: str = "Diffuser", ugate: np.array = UGATE) -> qc:
    """Amplify target state(s) amplitude, which decreases the amplitudes of other states
    and increases the probability of getting the correct solution (i.e. target state(s)).

    Args:
        name (str, optional): Quantum circuit's name. Defaults to "Diffuser".

    Returns:
        qc: Quantum circuit representation of diffuser (i.e. Grover's diffusion operator).
    """
    # Create N-qubit quantum circuit for diffuser
    diffuser = qc(QUBITS, name = name)
    
    diffuser.unitary(np.transpose(ugate), QUBITS)
    #diffuser.h(QUBITS)                                   
    diffuser.append(oracle({ "0" * N }), list(range(N)))  # Oracle with all zero target state
    diffuser.unitary(ugate, QUBITS)                                     
    #diffuser.h(QUBITS)     
    
    return diffuser

In [7]:
from math import pi, sqrt
from qiskit.quantum_info import DensityMatrix as dm

def grover(oracle: qc = oracle(), diffuser: qc = diffuser(), name: str = "Grover Circuit", ugate: np.array = UGATE) -> tuple[qc, dm]:
    """Create quantum circuit representation of Grover's algorithm,
    which consists of 4 parts: (1) state preparation/initialization,
    (2) oracle, (3) diffuser, and (4) measurement of resulting state.
    
    Steps 2-3 are repeated an optimal number of times (i.e. Grover's
    iterate) in order to maximize probability of success of Grover's algorithm.

    Args:
        oracle (qc, optional): Quantum circuit representation of oracle. Defaults to oracle().
        diffuser (qc, optional): Quantum circuit representation of diffuser. Defaults to diffuser().
        name (str, optional): Quantum circuit's name. Defaults to "Grover Circuit".

    Returns:
        tuple[qc, dm]: Quantum circuit representation of Grover's algorithm and its density matrix.
    """
    # Create N-qubit quantum circuit for Grover's algorithm
    grover = qc(QUBITS, name = name)

    # Intialize qubits with Hadamard gate (i.e. uniform superposition)
    grover.unitary(ugate, QUBITS)  
    
    # Apply barrier to separate steps
    grover.barrier()
    
    # Apply oracle and diffuser (i.e. Grover operator) optimal number of times
    num_itr = int((pi / 4) * sqrt((2 ** N) / len(TARGETS)))
    for _ in range(num_itr):
        grover.append(oracle, list(range(N)))
        grover.append(diffuser, list(range(N)))

    # Generate density matrix representation of Grover's algorithm
    density_matrix = dm(grover)

    # Measure all qubits once finished
    grover.measure_all()
    
    return grover, density_matrix

# (1) Generate and display quantum circuit for Grover's algorithm, and (2) Get density matrix
grover_oracle = oracle()
grover_diffuser = diffuser()
grover_circuit, density_matrix = grover(grover_oracle, grover_diffuser)

In [8]:
from heapq import nlargest
from qiskit import transpile
from qiskit_aer import AerSimulator

# Simulate Grover's algorithm locally
backend = AerSimulator(method = "density_matrix")

# Transpile circuit
transpiled = transpile(grover_circuit, backend, optimization_level = OPTIMIZATION_LEVEL)

# Run simulation 
simulation = backend.run(transpiled, shots = 10)
results = simulation.result().get_counts()
print(results)
#num_itr = int((pi / 4) * sqrt((2 ** N) / len(TARGETS)))
#step = 1

#while(not (list(results.keys())[0] == next(iter(TARGETS)))):
    #simulation = backend.run(transpiled, shots = 1)
    #results = simulation.result().get_counts()
    #step = step + 1

#print(step * num_itr)


{'00101': 10}
